In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import GridSearchCV

In [20]:
def extract_title(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    return "Unknown"


In [63]:
def format_df(data, fill=False):
    if fill:
        data = data.fillna(data.mean())
    data['Age'].fillna(data['Age'].mean(), inplace=True)
    data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)
    data['Fare'].fillna(data['Fare'].mean(), inplace=True)
    data['name_length']  = data['Name'].apply(len)

    label_encoder = LabelEncoder()
    data['Sex'] = label_encoder.fit_transform(data['Sex'])
    data['Embarked'] = label_encoder.fit_transform(data['Embarked'])
    data['Title'] = data['Name'].apply(extract_title)
    data['Title'] = label_encoder.fit_transform(data['Title'])

    data = data.drop(['Name', 'Ticket', 'Cabin'], axis=1)
    X = data.drop('Survived', axis=1)
    y = data['Survived']

    return X, y

In [65]:
df_val = pd.read_csv('test.csv')
df_val['Age'].fillna(df_val['Age'].mean(), inplace=True)
df_val['Embarked'].fillna(df_val['Embarked'].mode()[0], inplace=True)
df_val['Fare'].fillna(df_val['Fare'].mean(), inplace=True)
df_val['name_length']  = df_val['Name'].apply(len)

label_encoder = LabelEncoder()
df_val['Sex'] = label_encoder.fit_transform(df_val['Sex'])
df_val['Embarked'] = label_encoder.fit_transform(df_val['Embarked'])
df_val['Title'] = df_val['Name'].apply(extract_title)
df_val['Title'] = label_encoder.fit_transform(df_val['Title'])
df_val = df_val.drop(['Name', 'Ticket', 'Cabin'], axis=1)

y_val = pd.read_csv('gender_submission.csv')['Survived']

In [69]:
data = pd.read_csv("train.csv")
X, y = format_df(data)
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.05],
    'gamma': [0, 0.25, 1.0],
    'reg_lambda': [0, 1.0, 10.0],
    'scale_pos_weight': [1, 3, 5],
}
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xgb_clf  = xgb.XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss')
grid_clf = GridSearchCV(xgb_clf , param_grid, scoring='accuracy', cv=3, n_jobs=-1)

grid_clf.fit(X_train, y_train)

y_pred = grid_clf.predict(X_test)
print(f"Best parameters found: {grid_clf.best_params_}")
print(f"Best accuracy found: {grid_clf.best_score_}")

accuracy_score(grid_clf.predict(df_val), y_val)


Best parameters found: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'reg_lambda': 0, 'scale_pos_weight': 1}
Best accuracy found: 0.8342965878334456


0.8564593301435407

In [64]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [100, 200],
    'subsample': [1.0, 0.5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

clf = GradientBoostingClassifier()

data = pd.read_csv("train.csv")
X, y = format_df(data, fill=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
grid_clf = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_clf.fit(X_train, y_train)
y_pred = grid_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

C:\Users\Enzo\AppData\Local\Temp\ipykernel_19264\1119768689.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data = data.fillna(data.mean())


Fitting 5 folds for each of 324 candidates, totalling 1620 fits
0.8212290502793296


In [67]:
accuracy_score(grid_clf.predict(df_val), y_val)

0.8588516746411483